##  Geek News 카드뉴스 생성기 (Google Colab 버전)

이 노트북은 로컬 Python 프로젝트를 Google Colab 환경에서 실행할 수 있도록 변환한 것입니다.
아래 단계에 따라 실행하면 로컬 환경과 동일한 카드뉴스를 생성할 수 있습니다.

### 1단계: 초기 설정 (최초 1회만 실행)

카드뉴스 생성에 필요한 라이브러리를 설치하고, 각종 설정 파일과 폴더를 생성합니다.

**[중요]** 아래 셀을 실행하기 전에, Colab 왼쪽의 **'파일' 아이콘**을 클릭하여 **세션 저장소**에 필요한 파일과 폴더를 먼저 업로드해야 합니다.

1.  **`image` 폴더 업로드**: 로컬의 `image` 폴더를 Colab의 `/content/` 경로에 업로드하세요. 캐릭터 이미지가 여기에 포함됩니다.
2.  **`QR.png` 파일 업로드**: 로컬의 `QR.png` 파일을 `/content/` 경로에 업로드하세요.
3.  **Hugging Face 토큰 설정**: `HUGGINGFACE_TOKEN`을 Colab의 **'비밀' 아이콘** (열쇠 모양)에 추가하는 것을 권장합니다. 또는, 아래 `%%writefile .env` 코드에 직접 토큰을 입력해도 됩니다.

In [ ]:
# 1-1: 필요한 라이브러리 설치
!pip install -q transformers torch python-dotenv httpx beautifulsoup4 lxml playwright nest_asyncio

# 1-2: Playwright 브라우저 설치 (시간이 다소 걸릴 수 있습니다)
!playwright install

# 1-3: Colab에서 비동기 코드를 실행하기 위한 설정
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# 1-4: .env 파일 생성 (Hugging Face 토큰)
# [주의] 실제 토큰으로 바꿔주세요! (예: hf_xxxx...)
%%writefile .env
HUGGINGFACE_TOKEN=YOUR_HUGGINGFACE_TOKEN_HERE

In [ ]:
# 1-5: 프로젝트 폴더 구조 생성
import os

os.makedirs('templates', exist_ok=True)
os.makedirs('output/images', exist_ok=True)

In [ ]:
# 1-6: config.py 파일 생성
%%writefile config.py
# 파일 경로 설정
PATH_CONFIG = {
    "output_dir": "output",
    "template_dir": "templates",
    "style_file": "style.css",
    "cover_template": "cover_template.html",
    "news_template": "news_template.html",
    "summary_template": "summary_template.html",
    "summary_item_template": "summary_item_template.html",
    "summary_prompt": "summary_prompt.txt",
    "image_dir": "output/images",
    "character_dir": "image/character"
}

# 크롤링 설정
CRAWLING_CONFIG = {
    "news_count": 5,  # 가져올 뉴스 개수
    "base_url": "https://news.hada.io/",
    "timeout": 30  # HTTP 요청 타임아웃 (초)
}

# PDF/이미지 생성 설정
OUTPUT_CONFIG = {
    "page_width": 1080,
    "page_height": 1080,
    "pdf_margin": {'top': '0px', 'right': '0px', 'bottom': '0px', 'left': '0px'},
    "image_quality": 95,  # JPG 품질 (1-100)
    "generate_png": True,
    "generate_jpg": True,
    "generate_pdf": True
}

# 이미지 설정
IMAGE_CONFIG = {
    "character_extensions": [".png", ".jpg", ".jpeg"],
    "main_character": "1",
    "all_characters": "all"
}

# 색상 커스터마이징 설정
COLOR_CONFIG = {
    "cover_background": "linear-gradient(135deg, #E86E5A 0%, #D85745 100%)",    # 연한 빨간색 그라디언트
    "news_background": "linear-gradient(135deg, #E86E5A 0%, #D85745 100%)",     # 모든 페이지 통일
    "summary_background": "linear-gradient(135deg, #E86E5A 0%, #D85745 100%)",  # 모든 페이지 통일
    "end_background": "linear-gradient(135deg, #E86E5A 0%, #D85745 100%)"       # 모든 페이지 통일
}

# 폰트 크기 커스터마이징 설정
FONT_CONFIG = {
    "cover_title": "200px",
    "cover_subtitle": "50px",
    "news_title": "48px",        # 크게
    "news_description": "32px",   # 크게
    "news_category": "18px",
    "news_number": "36px",
    "link_text": "18px",         # 작게
    "summary_title": "72px",      # 원래대로
    "summary_subtitle": "36px",
    "summary_item_title": "22px"
}

# 텍스트 커스터마이징 설정
TEXT_CONFIG = {
    "cover_subtitle": "모여봐요 개발자와 AI의 숲",
    "cover_title": "",
    "news_card_prefix": "GeekNews",  # GeekNews #1, #2 형식
    "speech_bubble_text": "뉴-스!",
    "summary_title": "GeekNews 요약",
    "summary_subtitle": "오늘의 주요 뉴스",
    "summary_footer_text": "총 {count}개의 뉴스를 확인했어요",
    "summary_source": "출처: GeekNews (news.hada.io)"
}

# 이모지 커스터마이징 설정
EMOJI_CONFIG = {
    "speech_bubble": "💬",
    "lightbulb": "💡",
    "star": "⭐"
}

# AI 모델 설정
AI_CONFIG = {
    "model_name": "lcw99/t5-large-korean-text-summary",
    "max_input_length": 768,
    "max_output_length": 150,
    "min_output_length": 50,
    "length_penalty": 2.0,
    "num_beams": 4
}

In [ ]:
# 1-7: HTML & CSS 템플릿 파일 생성
%%writefile templates/style.css
body {
    margin: 0;
    padding: 0;
    font-family: 'Pretendard', -apple-system, BlinkMacSystemFont, system-ui, Roboto, 'Helvetica Neue', 'Segoe UI', 'Apple SD Gothic Neo', 'Noto Sans KR', 'Malgun Gothic', 'Apple Color Emoji', 'Segoe UI Emoji', 'Segoe UI Symbol', sans-serif;
    background: #000;
    color: #333;
    overflow-x: hidden;
}

a {
    color: {{cover_background}};
    text-decoration: none;
    transition: all 0.3s ease;
}

a:hover {
    color: #333;
    text-decoration: underline;
}


.page {
    width: 1080px;
    height: 1080px;
    display: flex;
    flex-direction: column;
    position: relative;
    page-break-after: always;
    box-sizing: border-box;
    overflow: hidden;
}

.card-container {
    width: 100%;
    height: 100%;
    padding: 60px;
    box-sizing: border-box;
    position: relative;
    display: flex;
    flex-direction: column;
}

/* 첫장 스타일 */
.cover {
    background: {{cover_background}};
}

.cover-container {
    position: relative;
    width: 100%;
    height: 100%;
    padding: 60px;
    box-sizing: border-box;
    display: flex;
    flex-direction: column;
    align-items: center;
    text-align: center;
}

.cover-header {
    margin-top: 120px;
}

.subtitle-small {
    font-size: 24px;
    color: rgba(255, 255, 255, 0.9);
    margin: 0;
    font-weight: 400;
    letter-spacing: 1px;
}

/* 뉴스 페이지 스타일 (모든 뉴스 페이지 동일) */
.news-page {
    background: {{news_background}};
}

/* 요약 페이지 스타일 */
.summary-page {
    background: {{summary_background}};
}

.summary-page .card-container {
    background: rgba(255, 255, 255, 0.12);
    border-radius: 30px;
    backdrop-filter: blur(22px);
    -webkit-backdrop-filter: blur(22px);
    border: 2px solid rgba(255, 255, 255, 0.35);
    box-shadow: 
        0 20px 45px rgba(0,0,0,0.35), 
        inset 0 0 45px rgba(255,255,255,0.12),
        inset 0 0 0 1px rgba(255,255,255,0.28); /* 내부 테두리 */
}

.cover-content {
    display: flex;
    flex-direction: column;
    align-items: center;
    gap: 40px;
    width: 100%;
}

.text-section {
    flex: 1;
    display: flex;
    flex-direction: column;
    justify-content: center;
    align-items: center;
    gap: 20px;
}

.title-sub {
    font-size: {{cover_subtitle_size}};
    color: rgba(255, 255, 255, 0.9);
    margin: 0;
    font-weight: 300;
    letter-spacing: 2px;
    text-shadow: 0 2px 10px rgba(0,0,0,0.3);
}

.cover-main {
    flex: 1;
    display: flex;
    flex-direction: column;
    justify-content: center;
    align-items: center;
}

.title-main {
    font-size: {{cover_title_size}};
    color: rgba(255, 255, 255, 0.9);
    margin: 0 0 40px 0;
    font-weight: 900;
    text-shadow: 0 8px 30px rgba(0,0,0,0.3);
    letter-spacing: -3px;
}

.character-section {
    flex: 1;
    display: flex;
    justify-content: center;
    align-items: center;
    position: relative;
}

.main-character {
    max-width: 250px;
    max-height: 250px;
    object-fit: contain;
    filter: drop-shadow(0 10px 25px rgba(0,0,0,0.3));
}

.qr-section {
    position: absolute;
    top: 60px;
    right: 60px;
}

.qr-code {
    width: 150px;
    height: 150px;
    background: white;
    padding: 10px;
    border-radius: 10px;
    box-shadow: 0 5px 15px rgba(0,0,0,0.2);
}

@keyframes float {
    0%, 100% { transform: translateY(0px); }
    50% { transform: translateY(-20px); }
}

.decorative-elements {
    position: absolute;
    width: 100%;
    height: 100%;
    pointer-events: none;
    overflow: hidden;
}

.speech-bubble {
    position: absolute;
    top: 15%;
    right: 15%;
    background: rgba(255, 255, 255, 0.95);
    padding: 15px 25px;
    border-radius: 25px;
    font-size: 24px;
    color: #333;
    box-shadow: 0 10px 25px rgba(0,0,0,0.2);
    animation: bounce 2s ease-in-out infinite;
}

.speech-bubble::after {
    content: '';
    position: absolute;
    bottom: -10px;
    left: 30px;
    width: 0;
    height: 0;
    border-left: 10px solid transparent;
    border-right: 10px solid transparent;
    border-top: 10px solid rgba(255, 255, 255, 0.95);
}

.bubble-text {
    font-weight: 700;
    margin-left: 10px;
}

@keyframes bounce {
    0%, 100% { transform: scale(1); }
    50% { transform: scale(1.1); }
}

.lightbulb {
    position: absolute;
    top: 20%;
    left: 10%;
    font-size: 40px;
    animation: glow 2s ease-in-out infinite alternate;
}

@keyframes glow {
    from { 
        text-shadow: 0 0 5px #ffeb3b, 0 0 10px #ffeb3b, 0 0 15px #ffeb3b;
        transform: rotate(-10deg);
    }
    to { 
        text-shadow: 0 0 10px #ffeb3b, 0 0 20px #ffeb3b, 0 0 30px #ffeb3b;
        transform: rotate(10deg);
    }
}

.star {
    position: absolute;
    font-size: 30px;
    color: #ffeb3b;
    animation: twinkle 1.5s ease-in-out infinite;
}

.star1 { top: 10%; left: 20%; animation-delay: 0s; }
.star2 { top: 30%; right: 20%; animation-delay: 0.5s; }
.star3 { bottom: 20%; left: 15%; animation-delay: 1s; }

@keyframes twinkle {
    0%, 100% { opacity: 0.3; transform: scale(1); }
    50% { opacity: 1; transform: scale(1.2); }
}

.news-header-section {
    margin-bottom: 40px;
    text-align: center;
}

.topic-category {
    display: inline-block;
    background: white;
    color: {{cover_background}};
    padding: 8px 20px;
    border-radius: 20px;
    font-size: {{news_category_size}};
    font-weight: 600;
    margin-bottom: 20px;
}

.category-title {
    font-size: {{news_number_size}};
    color: white;
    text-align: center;
    margin: 0;
    font-weight: 700;
}

.news-content-section {
    flex: 1;
    display: flex;
    flex-direction: column;
    gap: 30px;
}

.news-body {
    background: rgba(255, 255, 255, 0.1);
    border-radius: 20px;
    backdrop-filter: blur(18px);
    -webkit-backdrop-filter: blur(18px);
    padding: 50px;
    border-radius: 20px;
    flex: 1;
}

.news-title {
    font-size: {{news_title_size}};
    font-weight: 800;
    color: rgba(255, 255, 255, 0.8);
    text-decoration: underline;
    text-decoration-color: {{cover_background}};
    text-decoration-thickness: 2px;
    text-decoration-style: solid;
    text-underline-offset: 4px;
    text-align: center;
    line-height: 1.3;
    margin: 0 0 20px 0;
    word-break: keep-all;
}

.news-description {
    font-size: {{news_description_size}};
    line-height: 1.6;
    color: #333;
    margin: 0;
    word-break: keep-all;
}

.links-section {
    margin-top: auto;
    background: rgba(255, 255, 255, 0.3);
    padding: 20px;
    border-radius: 15px;
}

.link-item {
    margin-bottom: 12px;
    font-size: {{link_text_size}};
    color: #333;
}

.link-label {
    font-weight: 700;
    margin-right: 10px;
    color: #333;
}

.link-item a {
    color: {{cover_background}};
    text-decoration: none;
    word-break: break-all;
    transition: all 0.3s ease;
    font-weight: 500;
}

.link-item a:hover {
    color: #333;
    text-decoration: underline;
}

.page-character {
    position: absolute;
    bottom: 40px;
    right: 40px;
    max-width: 200px;
    max-height: 200px;
    object-fit: contain;
    filter: drop-shadow(0 8px 20px rgba(0,0,0,0.4));
    animation: float 4s ease-in-out infinite;
}

.summary-header-section {
    text-align: center;
    margin-bottom: 50px;
}

.summary-main-title {
    font-size: {{summary_title_size}};
    color: white;
    margin: 0 0 20px 0;
    font-weight: 900;
    text-shadow: 0 6px 25px rgba(0,0,0,0.5);
}

.summary-date {
    font-size: 24px;
    color: rgba(255, 255, 255, 0.9);
    margin: 0;
    font-weight: 300;
    text-shadow: 0 2px 10px rgba(0,0,0,0.3);
}

.summary-content {
    flex: 1;
}

.summary-subtitle {
    font-size: {{summary_subtitle_size}};
    color: white;
    margin: 0 0 30px 0;
    font-weight: 700;
    text-align: center;
    text-shadow: 0 4px 15px rgba(0,0,0,0.4);
}

.summary-list {
    display: flex;
    flex-direction: column;
    gap: 15px;
    margin-bottom: 20px;
}

.summary-item {
    background: rgba(255, 255, 255, 0.18);
    padding: 25px;
    border-radius: 20px;
    backdrop-filter: blur(18px);
    -webkit-backdrop-filter: blur(18px);
    border: 1px solid rgba(255, 255, 255, 0.3);
    box-shadow: 
        0 10px 25px rgba(0,0,0,0.3), 
        inset 0 0 20px rgba(255,255,255,0.1),
        inset 0 0 0 1px rgba(255,255,255,0.25); /* 내부 테두리 */
}

.summary-header {
    display: flex;
    align-items: center;
    gap: 15px;
    margin-bottom: 15px;
}

.summary-number {
    background: rgba(255, 255, 255, 0.9);
    color: #333;
    width: 40px;
    height: 40px;
    border-radius: 50%;
    display: flex;
    align-items: center;
    justify-content: center;
    font-weight: 700;
    font-size: 18px;
    box-shadow: 0 4px 10px rgba(0,0,0,0.2);
}

.summary-category {
    background: rgba(255, 255, 255, 0.25);
    color: white;
    padding: 6px 15px;
    border-radius: 15px;
    font-size: 14px;
    font-weight: 600;
    backdrop-filter: blur(10px);
    -webkit-backdrop-filter: blur(10px);
}

.summary-title {
    font-size: {{summary_item_title_size}};
    color: white;
    margin: 0;
    font-weight: 600;
    line-height: 1.4;
    text-shadow: 0 2px 10px rgba(0,0,0,0.4);
}

.summary-footer {
    text-align: center;
    padding-top: 30px;
    border-top: 2px solid rgba(255, 255, 255, 0.3);
}

.summary-footer p {
    color: rgba(255, 255, 255, 0.95);
    font-size: 20px;
    margin: 10px 0;
    font-weight: 500;
    text-shadow: 0 2px 8px rgba(0,0,0,0.3);
}

.summary-source {
    font-size: 18px !important;
    color: rgba(255, 255, 255, 0.8) !important;
    font-weight: 300 !important;
}

@media print {
    body { 
        margin: 0; 
        background: white;
    }
    .page { 
        page-break-after: always; 
        margin: 0;
        width: 1080px;
        height: 1080px;
    }
}


In [ ]:
%%writefile templates/cover_template.html
<div class="page cover">
    <div class="cover-container">
        <div class="cover-header">
            <p class="subtitle-small">{cover_subtitle}</p>
        </div>
        <div class="cover-main">
            <h1 class="title-main">{cover_title}</h1>
            <div class="character-section">
                {character_image}
            </div>
        </div>
        {qr_section}
    </div>
</div>

In [ ]:
%%writefile templates/news_template.html
<div class="page news-page">
    <div class="card-container">
        <div class="news-header-section">
            <div class="topic-category">{topic_category}</div>
            <h2 class="category-title">GeekNews #{page_number}</h2>
        </div>
        <div class="news-content-section">
            <div class="news-title">{news_title}</div>
            <div class="news-body">
                <div class="news-description">{news_description}</div>
            </div>
            <div class="links-section">
                {links}
            </div>
        </div>
        {character_image}
    </div>
</div>

In [ ]:
%%writefile templates/summary_template.html
<div class="page summary-page">
    <div class="card-container">
        <div class="summary-header-section">
            <h1 class="summary-main-title">{summary_title}</h1>
            <p class="summary-date">{summary_date}</p>
        </div>
        <div class="summary-content">
            <h2 class="summary-subtitle">{summary_subtitle}</h2>
            <div class="summary-list">
                {summary_items}
            </div>
            <div class="summary-footer">
                <p>{summary_footer}</p>
                <p class="summary-source">{summary_source}</p>
            </div>
        </div>
    </div>
</div>

In [ ]:
%%writefile templates/summary_item_template.html
<div class="summary-item">
    <div class="summary-header">
        <span class="summary-number">{number}</span>
        <span class="summary-category">{category}</span>
    </div>
    <div class="summary-title">{title}</div>
</div>

In [ ]:
%%writefile templates/summary_prompt.txt
다음 내용을 완전한 문장으로 요약하되, 부드럽고 친근한 문체로 작성해 주세요.

요약할 내용: {text}

요약은 모든 내용을 포함하게 기승전결로 작성해주세요


### 2단계: 카드뉴스 생성 실행

모든 설정이 완료되었습니다. 아래 셀을 실행하여 카드뉴스 생성을 시작합니다.

최종 결과물은 Colab의 `/content/output` 폴더에 PDF와 이미지 파일로 저장됩니다.

In [ ]:
# 메인 스크립트 실행
import os
import base64
from datetime import datetime
from bs4 import BeautifulSoup
from transformers import T5ForConditionalGeneration, AutoTokenizer
import torch
from config import *
from dotenv import load_dotenv
import httpx
from playwright.async_api import async_playwright
load_dotenv()
class GeekNewsCardGenerator:
    def __init__(self):
        self.output_dir = PATH_CONFIG["output_dir"]
        self.template_dir = PATH_CONFIG["template_dir"]
        print("한국어 요약 모델 로딩 중...")
        # .env 파일에서 Hugging Face 토큰 읽기
        hf_token = os.getenv('HUGGINGFACE_TOKEN')
        self.tokenizer = AutoTokenizer.from_pretrained(
            AI_CONFIG["model_name"],
            token=hf_token,
            trust_remote_code=True
        )
        self.model = T5ForConditionalGeneration.from_pretrained(
            AI_CONFIG["model_name"],
            token=hf_token
        )
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
        print("모델 로딩 완료")
        
    def summarize_text(self, text):
        if not text or len(text.strip()) < 50:
            print(f"  [요약 건너뜀] 텍스트가 너무 짧습니다: {text[:100]}")
            return ""
        
        print(f"  [요약 원문] {text[:150]}...")
        try:
            prompt_template = self.load_template(PATH_CONFIG["summary_prompt"])
            prompt = prompt_template.format(text=text)
            
            inputs = self.tokenizer(
                prompt,
                max_length=AI_CONFIG["max_input_length"], 
                truncation=True, 
                return_tensors="pt"
            ).to(self.device)
            summary_ids = self.model.generate(
                inputs["input_ids"],
                max_length=AI_CONFIG["max_output_length"],
                min_length=AI_CONFIG["min_output_length"],
                length_penalty=AI_CONFIG["length_penalty"],
                num_beams=AI_CONFIG["num_beams"],
                early_stopping=True
            )
            summary = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)
            print(f"  [요약 성공] {summary.strip()}")
            return summary.strip()
        except Exception as e:
            print(f"  [요약 실패] 오류: {e}")
            return "요약 생성에 실패했습니다."
    
    async def get_content(self, url):
        if not url:
            return ""
        try:
            print(f"  [디버그] 원문 기사 접근: {url}")
            async with httpx.AsyncClient() as client:
                response = await client.get(url, follow_redirects=True)
                soup = BeautifulSoup(response.text, 'lxml')
                if "github.com" in url:
                    content = soup.find('article')
                elif "youtube.com" in url or "youtu.be" in url:
                    return "" 
                else:
                    content = soup.find('body')
                if content:
                    content_text = content.get_text(separator=' ', strip=True)
                    print(f"  [디버그] 원문 내용 길이: {len(content_text)}")
                    return content_text
                return ""
        except Exception as e:
            print(f"  [오류] 기사 본문 가져오기 실패: {url}, {e}")
            return ""
    async def get_detail(self, topic_id):
        try:
            async with httpx.AsyncClient() as client:
                url = f'https://news.hada.io/topic?id={topic_id}'
                response = await client.get(url)
                soup = BeautifulSoup(response.text, 'lxml')
                # 긱뉴스 본문 내용 찾기 (class="topic_contents" 내부)
                contents_elem = soup.find('div', class_='topic_contents')
                if contents_elem:
                    return contents_elem.get_text(separator=' ', strip=True)
                # 대안으로 topic_desc 클래스도 확인
                desc_elem = soup.find('div', class_='topic_desc')
                if desc_elem:
                    return desc_elem.get_text(strip=True)
                return ""
        except Exception as e:
            print(f"긱뉴스 상세 정보 가져오기 실패: {e}")
            return ""
    async def fetch_geek_news(self):
        async with httpx.AsyncClient(timeout=CRAWLING_CONFIG["timeout"]) as client:
            response = await client.get(CRAWLING_CONFIG["base_url"])
            soup = BeautifulSoup(response.text, 'lxml')
            news_items = []
            topics = soup.find_all('div', class_='topic_row')[:CRAWLING_CONFIG["news_count"]]
            for topic in topics:
                title_elem = topic.find('div', class_='topictitle')
                if title_elem:
                    title_link = title_elem.find('a')
                    if title_link:
                        title = title_link.text.strip()
                        original_link = title_link.get('href', '')
                        # 긱뉴스 토픽 페이지 링크 찾기
                        topic_id = None
                        # 댓글 링크나 다른 내부 링크에서 topic ID 찾기
                        all_links = topic.find_all('a')
                        for link in all_links:
                            href = link.get('href', '')
                            if 'topic?id=' in href:
                                topic_id = href.split('id=')[-1].split('&')[0]
                                break
                        # 긱뉴스 페이지에서 설명 가져오기
                        desc_elem = topic.find('span', class_='topicdesc')
                        desc = desc_elem.text.strip() if desc_elem else ''
                        # 긱뉴스 토픽 페이지에서 본문 가져오기
                        if topic_id:
                            detailed_desc = await self.get_detail(topic_id)
                            if detailed_desc and len(detailed_desc) > len(desc):
                                desc = detailed_desc
                        # 설명 요약
                        summarized_desc = self.summarize_text(desc)
                        news_items.append({
                            'title': title,
                            'description': summarized_desc,
                            'link': original_link,
                            'topic_id': topic_id
                        })
            return news_items
    
    def generate_embedded_font_css(self):
        return """
@import url('https://fonts.googleapis.com/css2?family=Noto+Sans+KR:wght@300;400;500;700&display=swap');
@import url('https://fonts.googleapis.com/css2?family=Black+Han+Sans&display=swap');
"""
    def get_available_characters(self):
        characters = {}
        character_dir = PATH_CONFIG["character_dir"]
        if not os.path.exists(character_dir):
            return characters
        for file in os.listdir(character_dir):
            name, ext = os.path.splitext(file)
            if ext.lower() in [e.lower() for e in IMAGE_CONFIG["character_extensions"]]:
                characters[name] = os.path.join(character_dir, file)
        return characters
    def encode_image_to_base64(self, image_path):
        if not image_path or not os.path.exists(image_path):
            return None
        try:
            with open(image_path, "rb") as image_file:
                return base64.b64encode(image_file.read()).decode('utf-8')
        except Exception as e:
            print(f"이미지 인코딩 실패: {image_path}, 오류: {e}")
            return None
    def load_template(self, template_name):
        template_path = os.path.join(self.template_dir, template_name)
        return self.read_file(template_path)
    async def create_html(self, news_items):
        html_pages = ""
        
        available_characters = self.get_available_characters()
        main_character_path = available_characters.get(IMAGE_CONFIG["main_character"])
        main_character_base64 = self.encode_image_to_base64(main_character_path) if main_character_path else None
        
        page_characters = []
        for name, path in available_characters.items():
            if name != IMAGE_CONFIG["main_character"] and name != IMAGE_CONFIG["all_characters"]:
                character_base64 = self.encode_image_to_base64(path)
                if character_base64:
                    page_characters.append(character_base64)
        
        # QR 코드 로드 (루트 디렉토리에서)
        qr_path = "QR.png"
        qr_base64 = self.encode_image_to_base64(qr_path) if os.path.exists(qr_path) else None
        
        # 커버 템플릿 로드
        cover_template = self.load_template(PATH_CONFIG["cover_template"])
        
        character_html = f'<img src="data:image/png;base64,{main_character_base64}" class="character main-character" alt="캐릭터" />' if main_character_base64 else ''
        qr_html = f'<div class="qr-section"><img src="data:image/png;base64,{qr_base64}" class="qr-code" alt="QR코드" /></div>' if qr_base64 else ''
        
        html_pages += cover_template.format(
            cover_subtitle=TEXT_CONFIG["cover_subtitle"],
            cover_title=TEXT_CONFIG["cover_title"],
            character_image=character_html,
            qr_section=qr_html
        )
        
        # 뉴스 템플릿 로드
        news_template = self.load_template(PATH_CONFIG["news_template"])
        
        for page_index, news in enumerate(news_items):
            character_base64 = page_characters[page_index] if page_index < len(page_characters) else None
            
            domain_label = "링크"
            if news['link']:
                if "github.com" in news['link']:
                    domain_label = "GitHub"
                elif "youtube.com" in news['link'] or "youtu.be" in news['link']:
                    domain_label = "YouTube"
                elif "blog" in news['link'] or "medium.com" in news['link']:
                    domain_label = "블로그"
                elif "news" in news['link']:
                    domain_label = "뉴스"
                else:
                    domain_label = "원문"
            
            geek_news_link = f"https://news.hada.io/topic?id={news['topic_id']}" if news['topic_id'] else ""
            
            topic_category = "기술"
            if "github" in news['link'].lower() or "git" in news['title'].lower():
                topic_category = "개발"
            elif "youtube" in news['link'].lower():
                topic_category = "영상"
            elif "blog" in news['link'].lower():
                topic_category = "블로그"
            elif "ai" in news['title'].lower() or "gemini" in news['title'].lower():
                topic_category = "AI"
            elif "데이터" in news['title'] or "data" in news['title'].lower():
                topic_category = "데이터"
            
            # 링크 HTML 생성
            links_html = ""
            if geek_news_link:
                links_html += f'<div class="link-item"><span class="link-label">토론:</span> <a href="{geek_news_link}" target="_blank">{geek_news_link}</a></div>'
            if news['link']:
                links_html += f'<div class="link-item"><span class="link-label">{domain_label}:</span> <a href="{news["link"]}" target="_blank">{news["link"]}</a></div>'
            
            character_html = f'<img src="data:image/png;base64,{character_base64}" class="page-character" alt="캐릭터" />' if character_base64 else ''
            
            html_pages += news_template.format(
                topic_category=topic_category,
                page_number=page_index + 1,
                news_title=news['title'],
                news_description=news['description'],
                links=links_html,
                character_image=character_html
            )
        
        from datetime import datetime
        current_date = datetime.now().strftime("%Y년 %m월 %d일")
        
        # 요약 아이템 템플릿 로드
        summary_item_template = self.load_template(PATH_CONFIG["summary_item_template"])
        
        summary_items = ""
        for index, news in enumerate(news_items, 1):
            topic_category = "기술"
            if "github" in news['link'].lower() or "git" in news['title'].lower():
                topic_category = "개발"
            elif "ai" in news['title'].lower() or "gemini" in news['title'].lower():
                topic_category = "AI"
            elif "데이터" in news['title'] or "data" in news['title'].lower():
                topic_category = "데이터"
            
            summary_items += summary_item_template.format(
                number=index,
                category=topic_category,
                title=news['title']
            )
        
        # 요약 페이지 템플릿 로드
        summary_template = self.load_template(PATH_CONFIG["summary_template"])
        
        html_pages += summary_template.format(
            summary_title=TEXT_CONFIG["summary_title"],
            summary_date=current_date,
            summary_subtitle=TEXT_CONFIG["summary_subtitle"],
            summary_items=summary_items,
            summary_footer=TEXT_CONFIG["summary_footer_text"].format(count=len(news_items)),
            summary_source=TEXT_CONFIG["summary_source"]
        )
        
        return html_pages
    def create_styles(self):
        base_css = self.load_template(PATH_CONFIG["style_file"])
        
        # config.py의 설정으로 CSS 커스터마이징
        customized_css = base_css.replace("{{cover_background}}", COLOR_CONFIG["cover_background"])
        customized_css = customized_css.replace("{{news_background}}", COLOR_CONFIG["news_background"])
        customized_css = customized_css.replace("{{summary_background}}", COLOR_CONFIG["summary_background"])
        customized_css = customized_css.replace("{{end_background}}", COLOR_CONFIG["end_background"])
        
        customized_css = customized_css.replace("{{cover_title_size}}", FONT_CONFIG["cover_title"])
        customized_css = customized_css.replace("{{cover_subtitle_size}}", FONT_CONFIG["cover_subtitle"])
        customized_css = customized_css.replace("{{news_title_size}}", FONT_CONFIG["news_title"])
        customized_css = customized_css.replace("{{news_description_size}}", FONT_CONFIG["news_description"])
        customized_css = customized_css.replace("{{news_category_size}}", FONT_CONFIG["news_category"])
        customized_css = customized_css.replace("{{news_number_size}}", FONT_CONFIG["news_number"])
        customized_css = customized_css.replace("{{link_text_size}}", FONT_CONFIG["link_text"])
        customized_css = customized_css.replace("{{summary_title_size}}", FONT_CONFIG["summary_title"])
        customized_css = customized_css.replace("{{summary_subtitle_size}}", FONT_CONFIG["summary_subtitle"])
        customized_css = customized_css.replace("{{summary_item_title_size}}", FONT_CONFIG["summary_item_title"])
        
        return customized_css
    
    async def generate_all(self, html_content, css_content):
        embedded_fonts = self.generate_embedded_font_css()
        
        final_html = f"""
        <html>
        <head>
            <meta charset="UTF-8">
            <title>오늘의 긱뉴스</title>
            <style>
            {embedded_fonts}
            {css_content}
            </style>
        </head>
        <body>
            {html_content}
        </body>
        </html>
        """
        timestamp = datetime.now().strftime("%Y%m%d%H%M")
        pdf_filename = f"geek_news_{timestamp}.pdf"
        html_output_path = os.path.join(self.output_dir, "geek_news.html")
        pdf_output_path = os.path.join(self.output_dir, pdf_filename)
        with open(html_output_path, "w", encoding="utf-8") as f:
            f.write(final_html)
        print(f"'{html_output_path}' 파일 생성 완료")
        html_path_url = f'file://{os.path.abspath(html_output_path)}'
        async with async_playwright() as p:
            browser = await p.chromium.launch()
            page = await browser.new_page()
            await page.set_viewport_size({"width": OUTPUT_CONFIG["page_width"], "height": OUTPUT_CONFIG["page_height"]})
            await page.goto(html_path_url, wait_until='networkidle')
            await page.wait_for_timeout(2000)
            if OUTPUT_CONFIG["generate_pdf"]:
                await page.pdf(
                    path=pdf_output_path, 
                    width=f"{OUTPUT_CONFIG['page_width']}px", 
                    height=f"{OUTPUT_CONFIG['page_height']}px", 
                    print_background=True,
                    margin=OUTPUT_CONFIG["pdf_margin"]
                )
                print(f"'{pdf_output_path}' 파일 생성 완료")
            pages = await page.query_selector_all('.page')
            print(f"총 {len(pages)} 페이지 이미지 생성 중...")
            for i, page_element in enumerate(pages, 1):
                if OUTPUT_CONFIG["generate_png"]:
                    png_filename = f"geek_page_{i:02d}.png"
                    png_output_path = os.path.join(PATH_CONFIG["image_dir"], png_filename)
                    await page_element.screenshot(
                        path=png_output_path,
                        type='png',
                        omit_background=False
                    )
                    print(f"'{png_filename}' 생성 완료")
                    
                if OUTPUT_CONFIG["generate_jpg"]:
                    jpg_filename = f"geek_page_{i:02d}.jpg"
                    jpg_output_path = os.path.join(PATH_CONFIG["image_dir"], jpg_filename)
                    await page_element.screenshot(
                        path=jpg_output_path,
                        type='jpeg',
                        quality=OUTPUT_CONFIG["image_quality"],
                        omit_background=False
                    )
                    print(f"'{jpg_filename}' 생성 완료")
            
            await browser.close()
    
    def read_file(self, filepath):
        try:
            with open(filepath, "r", encoding="utf-8") as f:
                return f.read()
        except FileNotFoundError:
            print(f"오류: 파일 '{filepath}'을(를) 찾을 수 없습니다.")
            return ""
    
    def create_output_directory(self):
        os.makedirs(self.output_dir, exist_ok=True)
        os.makedirs(PATH_CONFIG["image_dir"], exist_ok=True)
    
    async def generate(self):
        print("=== 긱뉴스 카드뉴스 생성 시작 ===")
        self.create_output_directory()
        news_items = await self.fetch_geek_news()
        if not news_items:
            print("뉴스를 가져올 수 없습니다.")
            return
        print(f"{len(news_items)}개의 뉴스를 가져왔습니다.")
        html_content = await self.create_html(news_items)
        css_content = self.create_styles()
        await self.generate_all(html_content, css_content)
        print("=== 긱뉴스 카드뉴스 생성 완료 ===")

# 메인 실행 함수
async def main():
    generator = GeekNewsCardGenerator()
    await generator.generate()

# 스크립트 실행
await main()
